In this notebook we'll fetch the urls of the companies that are reference on trustpilot.com

We'll use selenium because the content is dynamically rendered

We'll then scrape the reviews using scrapy amd feeding it the scraped urls

In [1]:
%config Completer.use_jedi=False

In [2]:
import json
import time

from bs4 import BeautifulSoup
import requests
import pandas as pd

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

from tqdm import tqdm_notebook
print('success')

success


In [3]:
base_url = "https://trustpilot.com"

In [4]:
def get_soup(url):
    return BeautifulSoup(requests.get(url).content, 'lxml')
print('success')

success


We first start by fetching sub-categories urls:

In [5]:
data = {}

soup = get_soup(base_url + '/categories')
for category in soup.findAll('div', {'class' : 'category-object'}):
    name = category.find('h3', {'class' : 'sub-category__header'}).text
    name = name.strip()
    data[name] = {}
    sub_categories = category.find('div', {'class' : 'sub-category-list'})
    for sub_category in sub_categories.findAll('div', {'class' : 'child_category'}):
        sub_category_name = sub_category.find('a', {'class' : 'sub-category-item'}).text
        sub_category_uri = sub_category.find('a', {'class' : 'sub-category-item'})['href']
        data[name][sub_category_name] = sub_category_uri
print('success')

success


In [6]:
len(data)

22

This function allows to fetch company urls reference in a given subcategory:

In [7]:
def extract_company_urls_form_page():
    a_list = driver.find_elements_by_xpath('//a[@class="category-business-card card"]')
    urls = [a.get_attribute('href') for a in a_list]
    dedup_urls = list(set(urls))
    return dedup_urls

This functions indicates if a pagination exists on the page:

In [8]:
def go_next_page():
    try:
        button = driver.find_element_by_xpath('//a[@class="button button--primary next-page"]')
        return True, button
    except NoSuchElementException:
            return False, None

We start by initializing Selenium with a headless Chromedriver:

In [14]:
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('start-maximized')
options.add_argument('disable-infobars')
options.add_argument("--disable-extensions")

prefs = {"profile.managed_default_content_settings.images" : 2}
options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(r'C:\Users\isaama2\Documents\Chromedriver\chromedriver.exe', options=options)

timeout = 3

The timeout variable is the time (in seconds) Selenium waits for a page to completely load.

Now we launch the scraping. This approcimately takes 50 minutes with good internet conntection.

In [ ]:
company_urls = {}

for category in tqdm_notebook(data):
    for sub_category in tqdm_notebook(data[category], leave = False):
        company_urls[sub_category] = []